# Библиотеки

In [1]:
import numpy as np
import sympy as smp

# Классы для производных

In [2]:
class DerivativeNum:
    
    def __init__(self, function=None, h=None, alpha=1, dots=0):
        self._function = function
        self._h = h
        self.SetAlpha(alpha)
        self.SetDots(dots)

    def SetAlpha(self, alpha):
        self._alpha = alpha
        self._alphaN = alpha
        
    def SetDots(self, dots):
        self._dots = dots
        self._dotsN = dots
    
    def Calculate(self, x):
        return np.dot(self._alphaN, self._function(x+self._h*self._dotsN))/(self._h**self._n)

    def __call__(self, x):
        return self.Calculate(x)

In [3]:
class DerivativeNum4(DerivativeNum):
    def __init__(self, function=None, h=None):
        super().__init__(function, h, np.array([0.5, -0.5]), np.array([1., -1.]))
        self._n = 1

In [4]:
class DerivativeNNum(DerivativeNum4):
    def __init__(self, n=0, function=None, h=None):
        super().__init__(function=function, h=h)
    
    def SetN(self, n):
        self._n = n
        self._alphaN = np.array([])
        self._dotsN = np.array([])
        x, y = smp.symbols('x y')
        self._scheme = smp.Poly((self._alpha[0]*x+self._alpha[1]*y)**n, x, y).as_dict().items()
        for key, value in self._scheme:
            self._alphaN = np.append(self._alphaN, value)
            self._dotsN = np.append(self._dotsN, np.array(np.dot(key, self._dots)))
            
    def __call__(self, x):
        return self.Calculate(x)

# Классы для нормированных пространств

In [5]:
class NormedVectorSpace:
    def __init__(self, left=None, right=None, function=None, h=None, *args):
        super().__init__()
        self.SetLeft(left)
        self.SetRight(right)
        self.SetFunction(function)
        self.SetH(h)
    def SetLeft(self, left):
        self._left = left
    def SetRight(self, right):
        self._right = right
    def SetFunction(self, function):
        self._function = function
    def SetH(self, h):
        self._h = h

In [6]:
class NormedVectorSpaceCN(NormedVectorSpace, DerivativeNNum):
    def __init__(self, left=0, right=2, function=None, h=1):
        super().__init__(left=left, right=right, function=function, h=h)
        self.SetFunction(function)
        self.SetH(h)
        
    def GetNormN(self, n):
        self.SetN(n)
        z = np.amax([abs(self(x)) for x in np.arange(self._left, self._right, self._h)])
        return z
    
    def CalculateNorm(self, n=None):
        answer = 0
        if n is None:
            n=self._n
            print('C%s :' % n)
        for i in range(n+1):
            answer += self.GetNormN(i)
        return answer
    
    def CalculateMetric(self, f1, f2, n=None):
        self._function = smp.lambdify(xSym, f1-f2)
        return self.CalculateNorm(n)

# Исследуемые функции

In [7]:
xSym = smp.Symbol('x')
functionsSym = [5/(2+3*xSym**2),
                2/(5+smp.cos(xSym)),
                (3+4*xSym**2)**(1/3),
                2/smp.pi**(1/2)*smp.exp(-xSym**2)
               ]
functions = [smp.lambdify(xSym, fSym) for fSym in functionsSym]

# Нормы функций

In [8]:
CN = NormedVectorSpaceCN(h=1e-3, left=0, right=2)
for index, f in enumerate(functions):
    CN.SetFunction(f)
    print('Function is %s' % functionsSym[index])
    for i in range(3):
        print('    C%s Norm: %s' % (i, CN.CalculateNorm(i)))

Function is 5/(3*x**2 + 2)
    C0 Norm: 2.50000000000000
    C1 Norm: 4.48873448195858
    C2 Norm: 11.9886894821150
Function is 2/(cos(x) + 5)
    C0 Norm: 0.436227596601593
    C1 Norm: 0.522903841717501
    C2 Norm: 0.578459397290074
Function is (4*x**2 + 3)**0.333333333333333
    C0 Norm: 2.66765260070813
    C1 Norm: 3.43079540081238
    C2 Norm: 4.71279273976567
Function is 2*pi**(-0.5)*exp(-x**2)
    C0 Norm: 1.12837916709551
    C1 Norm: 2.09626139774643
    C2 Norm: 4.35301521845277


# Попарные расстояния

In [9]:
CN = NormedVectorSpaceCN(h=1e-3, left=0, right=2)
for index1, f1 in enumerate(functionsSym):
    for index2, f2 in enumerate(functionsSym):
        if index1>=index2:
            continue
        print('Pair functions are %s and %s' % (f1, f2))
        for i in range(3):
            print('    C%s Metrics: %s' % (i , CN.CalculateMetric(f1,f2,n=i)))

Pair functions are 5/(3*x**2 + 2) and 2/(cos(x) + 5)
    C0 Metrics: 2.16666666666667
    C1 Metrics: 4.18168874480286
    C2 Metrics: 11.7371993004486
Pair functions are 5/(3*x**2 + 2) and (4*x**2 + 3)**0.333333333333333
    C0 Metrics: 2.31020343516248
    C1 Metrics: 4.82744631751104
    C2 Metrics: 13.6093986566208
Pair functions are 5/(3*x**2 + 2) and 2*pi**(-0.5)*exp(-x**2)
    C0 Metrics: 1.37162083290449
    C1 Metrics: 2.55505207761525
    C2 Metrics: 7.79825325706535
Pair functions are 2/(cos(x) + 5) and (4*x**2 + 3)**0.333333333333333
    C0 Metrics: 2.23142500410653
    C1 Metrics: 2.92103551258258
    C2 Metrics: 4.14747729593554
Pair functions are 2/(cos(x) + 5) and 2*pi**(-0.5)*exp(-x**2)
    C0 Metrics: 0.795045833762179
    C1 Metrics: 1.80247364616641
    C2 Metrics: 4.11478302247308
Pair functions are (4*x**2 + 3)**0.333333333333333 and 2*pi**(-0.5)*exp(-x**2)
    C0 Metrics: 2.64690280260587
    C1 Metrics: 4.27862138309892
    C2 Metrics: 7.81737254275855
